In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210705-1430-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
#variant_qc/models/5ad0be9c/rf_result_MegaWES_new.ht
#d1d1ec80
#ch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/17cce417/rf_result_MegaWES_new.ht

ht=hl.read_table(f'{lustre_dir}/variant_qc/models/91ba5f38/rf_result_MegaWES_new.ht')

In [3]:
ht.count()

28803842

In [4]:
ht2=ht.filter(hl.is_missing(ht.rf_probability['TP']))

In [5]:
ht2.count()

0

In [6]:
ht1=ht.filter(hl.is_defined(ht.rf_probability['TP']))

In [7]:
ht1.count()

28803842

In [4]:
ht.count()

28803842

In [5]:
ht.describe()

----------------------------------------
Global fields:
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'MQ': float64 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': float64 
    'FS': float64 
    'DP': int32 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'ac_raw': int64 
    'tp': bool 
    'fp': bool 
    'rf_train': bool 
    'rf_label': str 
    'rf_test': bool 
    'rf_pro

In [5]:
ht.rf_probability.describe()

--------------------------------------------------------
Type:
        dict<str, float64>
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [6]:
ht1=ht.filter(hl.is_defined(ht.rf_probability['TP']))

In [7]:
ht2=ht.filter(hl.is_missing(ht.rf_probability['TP']))

In [8]:
ht2.count()

7969027

In [9]:
ht1.count()

20834815

In [10]:
ht2.summarize()

Non-missing,7969027 (100.00%)
Missing,0
Contig Counts,"{'chr11': 427097, 'chr5': 417513, 'chr22': 178307, 'chr8': 338118, 'chr19': 333354, 'chrY': 16208, 'chr1': 729419, 'chr15': 321720, 'chr12': 460630, 'chr18': 206701, 'chr20': 197329, 'chr2': 591087, 'chr13': 198510, 'chr7': 377941, 'chr14': 297038, 'chr3': 476707, 'chrM': 556, 'chr17': 374967, 'chr4': 369398, 'chr6': 385400, 'chr9': 276614, 'chrX': 245092, 'chr10': 311772, 'chr21': 106766, 'chr16': 330783}"
Non-missing,7969027 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,15938054 (100.00%)
Missing,0
Min Size,1


In [11]:
ht1.summarize()

Non-missing,20834815 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1205100, 'chr5': 925102, 'chr22': 517840, 'chr8': 766444, 'chr19': 1328730, 'chrY': 12496, 'chr1': 2030146, 'chr15': 756175, 'chr12': 1111124, 'chr18': 356431, 'chr20': 523669, 'chr2': 1508590, 'chr13': 397670, 'chr7': 1053807, 'chr14': 694378, 'chr3': 1177096, 'chrM': 6512, 'chr17': 1180709, 'chr4': 831282, 'chr6': 975006, 'chr9': 882970, 'chrX': 514935, 'chr10': 836069, 'chr21': 247523, 'chr16': 995011}"
Non-missing,20834815 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,41669630 (100.00%)
Missing,0
Min Size,1


In [7]:
ht1.write(f'{lustre_dir}/variant_qc/models/5ad0be9c/rf_result_MegaWES_new_non_missing.ht')

2021-07-02 08:36:13 Hail: INFO: wrote table with 17592119 rows in 487 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/5ad0be9c/rf_result_MegaWES_new_non_missing.ht
    Total size: 976.64 MiB
    * Rows: 976.64 MiB
    * Globals: 308.00 B
    * Smallest partition: 7644 rows (442.35 KiB)
    * Largest partition:  41941 rows (2.53 MiB)


In [2]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/1a6950e6/rf_result_MegaWES_new.ht')

In [3]:
ht.count()

28803842

In [4]:
ht.describe()

----------------------------------------
Global fields:
    'test_intervals': array<interval<locus<GRCh38>>> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'MQ': float64 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': float64 
    'FS': float64 
    'DP': int32 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'ac_raw': int64 
    'tp': bool 
    'fp': bool 
    'rf_train': bool 

In [2]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/1a6950e6_rf_result_FINAL_for_RANKING.ht')

In [3]:
ht.count()

28803842

In [4]:
ht.describe()

----------------------------------------
Global fields:
    'test_intervals': array<interval<locus<GRCh38>>> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'MQ': float64 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': float64 
    'FS': float64 
    'DP': int32 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'ac_raw': int64 
    'tp': bool 
    'fp': bool 
    'rf_train': bool 

In [6]:
ht.variant_untransmitted_singletons.summarize()

Non-missing,890975 (3.09%)
Missing,27912867 (96.91%)
Minimum,0
Maximum,5
Mean,0.11
Std Dev,0.32


In [7]:
ht.variant_transmitted_singletons.summarize()

Non-missing,890975 (3.09%)
Missing,27912867 (96.91%)
Minimum,0
Maximum,5
Mean,0.13
Std Dev,0.35
